In [1]:
import pandas as pd

cellphone_data = pd.read_json("/content/Cell_Phones_and_Accessories_5.json", lines=True)

cellphone_data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


Data preprocessing

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer("[a-zA-Z0-9]+")
vectorizer = CountVectorizer( stop_words='english', tokenizer = regexp.tokenize )


In [4]:
cellphone_data['reviewText'].isnull().any()

False

In [5]:
cellphone_data['overall'].isnull().any()

False

In [6]:
cellphone_data['summary'].isnull().any()

False

In [10]:
X = vectorizer.fit_transform( cellphone_data['reviewText'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, cellphone_data['overall'], test_size = 0.25)

In [13]:
cellphone_data['overall'].dtypes

dtype('int64')

In [14]:
cellphone_data['overall'].value_counts

<bound method IndexOpsMixin.value_counts of 0         4
1         5
2         5
3         4
4         5
         ..
194434    5
194435    5
194436    5
194437    5
194438    5
Name: overall, Length: 194439, dtype: int64>

In [16]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()

In [17]:
mnb.fit(X_train, y_train)

MultinomialNB()

In [18]:
y_pred = mnb.predict( X_test )

In [19]:
y_pred

array([5, 5, 5, ..., 3, 5, 4])

In [20]:
from sklearn.metrics import accuracy_score

acc = accuracy_score( y_test, y_pred )

print( acc )

0.6065212919152437


Thus the accuracy is very poor around 60 percent when building the model based on the review text.

Summary VS Overall Score

In [22]:
input_summ = vectorizer.fit_transform( cellphone_data['summary'])

X_train_summ, X_test_summ, y_train_summ, y_test_summ = train_test_split( input_summ, cellphone_data['overall'], test_size = 0.25 )

mnb_summ = MultinomialNB()

mnb_summ.fit( X_train_summ, y_train_summ )

y_pred_summ = mnb_summ.predict( X_test_summ )

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [23]:
acc_summ = accuracy_score( y_pred_summ, y_test )

print(f"The accuracy when Summary vs Overral score is {acc_summ}")

The accuracy when Summary vs Overral score is 0.4634437358568196


We find the score to be even more poorer with Summary and Overall score. Hence we will use the Review Text vs Overall scores as the preferred variable.